In [ ]:
# test jupy nb for data analysis and preprocessing big creators
import pandas as pd
from collections import Counter
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

df = pd.read_csv("ytraw.csv")
print(df.head())

      video_id                                              title  \
0  ISSjUbAVJa0                         Okay fine, I'll try Skyrim   
1  ST5mY0HE_7o  SKYRIM Full Movie 2025: Oblivion | FullHDvideo...   
2  V2OoTDk9JiI  These NPCs don’t know they’re talking to the D...   
3  2AbEyr8nIcA  "THAT" Jumpscared Me 💀 #skyrim #skyrimannivers...   
4  B-oe9ypNSb0  New Skyrim Mods You Should Try Before Starting...   

                                         description  \
0  Skyrim.. the game everyone says is their favor...   
1  Skyrim Full Movie 2025: Oblivion | FullHDvideo...   
2                                                NaN   
3  #Skyrim #Anniversary #skyrimae \n\nThe Elder S...   
4  🔗 https://ene.ba/MasterCheesey\n🎮 https://ene....   

                 channel_id    channel_title  category_id    views   likes  \
0  UCO9US9tHbfyTD3eGJEbD1Ig             Bind           22  2238569   75971   
1  UCpiLRBQAIDhzuxnDi9Oar4A  FullHDvideos4me            1  2246428   12513   
2  UCaHAsSOlU3

In [2]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   video_id       300 non-null    object
 1   title          300 non-null    object
 2   description    265 non-null    object
 3   channel_id     300 non-null    object
 4   channel_title  300 non-null    object
 5   category_id    300 non-null    int64 
 6   views          300 non-null    int64 
 7   likes          300 non-null    int64 
 8   comments       300 non-null    int64 
 9   duration       300 non-null    object
 10  published_at   300 non-null    object
 11  keyword        300 non-null    object
dtypes: int64(4), object(8)
memory usage: 28.3+ KB
None


In [3]:
print(df.describe())

       category_id         views         likes      comments
count   300.000000  3.000000e+02  3.000000e+02    300.000000
mean     19.676667  3.457251e+06  9.100903e+04   1117.073333
std       3.711879  8.753740e+06  1.975987e+05   2412.616779
min       1.000000  8.900000e+01  0.000000e+00      0.000000
25%      20.000000  1.770268e+05  5.986250e+03    128.500000
50%      20.000000  8.810815e+05  2.567850e+04    416.500000
75%      20.000000  2.635452e+06  8.419175e+04   1079.250000
max      27.000000  8.307708e+07  2.039668e+06  28234.000000


,video_id,title,description,channel_id,channel_title,category_id,views,likes,comments,duration,published_at,keyword
0,ISSjUbAVJa0,"Okay fine, I'll try Skyrim",Skyrim.. the game everyone says is their favor...,UCO9US9tHbfyTD3eGJEbD1Ig,Bind,22,2238569,75971,7808,PT38M23S,2025-04-14T20:00:58Z,skyrim
1,ST5mY0HE_7o,SKYRIM Full Movie 2025: Oblivion | FullHDvideo...,Skyrim Full Movie 2025: Oblivion | FullHDvideo...,UCpiLRBQAIDhzuxnDi9Oar4A,FullHDvideos4me,1,2246428,12513,236,PT2H28S,2025-05-03T09:00:38Z,skyrim
2,V2OoTDk9JiI,These NPCs don’t know they’re talking to the D...,,UCaHAsSOlU3yuYxohUrMVtBA,Skooma Sippa,20,4716091,161780,3087,PT36S,2025-08-28T17:14:52Z,skyrim
3,2AbEyr8nIcA,"""THAT"" Jumpscared Me 💀 #skyrim #skyrimannivers...",#Skyrim #Anniversary #skyrimae \n\nThe Elder S...,UCnVbLunqMpiik_WEA3dSVaw,Salmagros,20,4069162,106650,3046,PT7S,2024-09-09T06:02:45Z,skyrim
4,B-oe9ypNSb0,New Skyrim Mods You Should Try Before Starting...,🔗 https://ene.ba/MasterCheesey\n🎮 https://ene....,UCjpUc0ZOzwHYkFIs3U4Z9fg,Master Cheesey,20,3562,288,33,PT11M33S,2025-09-11T16:00:18Z,skyrim
